<a href="https://colab.research.google.com/github/yachtsman/Coursera_Capstone/blob/master/Segmenting_and_Clustering_Neighborhoods_in_Toronto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Segmenting and Clustering Neighborhoods in Toronto

# Mining and structuring the data

1.   Creating a new notebook
2.   Scraping data from Wikipedia
3.   Creating the dataframe
4.   Submiting the link on Github





1. Creating a new notebook

Importing basic dependencies

In [1]:
import pandas as pd
import numpy as np
print('basic dependencies imported')

basic dependencies imported


2. Scraping data from wikipedia

In [2]:
from bs4 import BeautifulSoup
import requests
#from urllib.request import urlopen
#import csv

print('BeautifulSoup is imported.')

BeautifulSoup is imported.


In [3]:
wiki_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(wiki_url,'lxml')
print('soup cooked')

soup cooked


In [4]:
wiki_table = soup.find('table',{'class':'wikitable sortable'})
wiki_rows = wiki_table.find_all('tr')
#wiki_rows

3. Creating the dataframe

In [6]:
data = []
for row in wiki_rows:
    data.append([t.text.strip()
    for t in row.find_all('td')]
    )
df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df.drop([0], inplace=True) # to skip first rows 'none' 
print(df.head())
print(df.shape)

  PostalCode           Borough              Neighbourhood
1        M1A      Not assigned               Not assigned
2        M2A      Not assigned               Not assigned
3        M3A        North York                  Parkwoods
4        M4A        North York           Victoria Village
5        M5A  Downtown Toronto  Regent Park, Harbourfront
(180, 3)


Cleaning  the 'Not assigned' rows

In [7]:
df1 = df[df['Borough'] != 'Not assigned'].reset_index(drop=True) # to filter out 'Not assigned' rows
print(df1.head())
print(df1.shape)

  PostalCode           Borough                                Neighbourhood
0        M3A        North York                                    Parkwoods
1        M4A        North York                             Victoria Village
2        M5A  Downtown Toronto                    Regent Park, Harbourfront
3        M6A        North York             Lawrence Manor, Lawrence Heights
4        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government
(103, 3)


combinig the Neighborhoods with the equal postal codes 

In [8]:
df2= df1.groupby('PostalCode').agg(lambda x: ','.join(x))
print(df2.head())
print(df2.shape)

                Borough                           Neighbourhood
PostalCode                                                     
M1B         Scarborough                          Malvern, Rouge
M1C         Scarborough  Rouge Hill, Port Union, Highland Creek
M1E         Scarborough       Guildwood, Morningside, West Hill
M1G         Scarborough                                  Woburn
M1H         Scarborough                               Cedarbrae
(103, 2)


In [9]:
df2.loc[df2['Neighbourhood']=="Not assigned",'Neighbourhood']=df2.loc[df2['Neighbourhood']=="Not assigned",'Borough']
df3 = df2.reset_index()

In [10]:
df3['Borough']= df3['Borough'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")
print('The final cleaned table looks like:')
print(df3.head(20))
print('with the shape:')
print(df3.shape)

The final cleaned table looks like:
   PostalCode      Borough                                      Neighbourhood
0         M1B  Scarborough                                     Malvern, Rouge
1         M1C  Scarborough             Rouge Hill, Port Union, Highland Creek
2         M1E  Scarborough                  Guildwood, Morningside, West Hill
3         M1G  Scarborough                                             Woburn
4         M1H  Scarborough                                          Cedarbrae
5         M1J  Scarborough                                Scarborough Village
6         M1K  Scarborough        Kennedy Park, Ionview, East Birchmount Park
7         M1L  Scarborough                    Golden Mile, Clairlea, Oakridge
8         M1M  Scarborough    Cliffside, Cliffcrest, Scarborough Village West
9         M1N  Scarborough                        Birch Cliff, Cliffside West
10        M1P  Scarborough  Dorset Park, Wexford Heights, Scarborough Town...
11        M1R  Scarborough  

In [16]:
df3

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


# Adding the latitude and the longitude coordinates of each neighborhood

In [12]:
csv_path = 'https://cocl.us/Geospatial_data'
df4 = pd.read_csv(csv_path)

In [13]:
print(df4.head(20))
print(df4.shape)

   Postal Code   Latitude  Longitude
0          M1B  43.806686 -79.194353
1          M1C  43.784535 -79.160497
2          M1E  43.763573 -79.188711
3          M1G  43.770992 -79.216917
4          M1H  43.773136 -79.239476
5          M1J  43.744734 -79.239476
6          M1K  43.727929 -79.262029
7          M1L  43.711112 -79.284577
8          M1M  43.716316 -79.239476
9          M1N  43.692657 -79.264848
10         M1P  43.757410 -79.273304
11         M1R  43.750072 -79.295849
12         M1S  43.794200 -79.262029
13         M1T  43.781638 -79.304302
14         M1V  43.815252 -79.284577
15         M1W  43.799525 -79.318389
16         M1X  43.836125 -79.205636
17         M2H  43.803762 -79.363452
18         M2J  43.778517 -79.346556
19         M2K  43.786947 -79.385975
(103, 3)


In [15]:
df4.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
df4.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [23]:
df_geo = pd.merge(df3, df4, how='inner')

df_geo.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
